# Orbit Calibration Set Up  
This notebook will download the relevant data from space-track from the sp3 files that you provide. This will be then stored in the rawdata folder that can then be used for the calibration analysis. 

In [1]:
import os

ModuleNotFoundError: No module named 'sp3'

In [5]:
# manually fill out the folder where your s3 data for grace-fo-1 and grace-fo-2 is stored
RawDataFolderName = 'v1'

# First check how many files that have the *.sp3 extension, if more than 2 give a warning to the user

# get the current working directory
cwd = os.getcwd()
RawDataFolder = os.path.join(cwd, 'RawData', RawDataFolderName)

# get all the files in the RawDataFolder
files = os.listdir(RawDataFolder)

# check the number with the *.sp3 extension
sp3files = [file for file in files if file.endswith('.sp3')]
if len(sp3files) != 2:
    print('Warning: There are {} files with the *.sp3 extension in the RawData folder. There should be 2.'.format(len(sp3files)))

# grace1 is the one that has L64 in the file name
grace1Name = [file for file in sp3files if 'L64' in file][0]
grace2Name = [file for file in sp3files if 'L65' in file][0]

'GFZOP_RSO_L64_G_20230924_100000_20230925_000000_v03.sp3'